In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on
# %pycodestyle_off -to turn it off

In [2]:
from minio import Minio
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, month, year
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, LongType
import requests
import json
import pandas as pd
from binance import Client
from datetime import datetime, date
import os
from dotenv import load_dotenv, dotenv_values
from binance.helpers import date_to_milliseconds, interval_to_milliseconds
from binance.exceptions import BinanceRequestException, BinanceAPIException
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
import time
import logging
load_dotenv()

True

5:80: E501 line too long (87 > 79 characters)


In [3]:
API_KEY = os.getenv("API_KEY")
SECRET_KEY = os.getenv("SECRET_KEY")
MINIO_USER = os.getenv("MINIO_ROOT_USER")
MINIO_PASSWORD = os.getenv("MINIO_ROOT_PASSWORD")
client_binance = Client(API_KEY, SECRET_KEY)

In [4]:
spark = SparkSession.builder \
    .appName("CryptoETL") \
    .getOrCreate()
# Get the SparkContext from the SparkSession
sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", MINIO_USER)#turn into access key in the future 
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", MINIO_PASSWORD)
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://localhost:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.attempts.maximum", "1")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.establish.timeout", "5000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.timeout", "10000")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/17 16:45:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
6:67: E261 at least two spaces before inline comment
6:67: E262 inline comment should start with '# '
6:80: E501 line too long (101 > 79 characters)
6:102: W291 trailing whitespace
12:80: E501 line too long (80 > 79 characters)


In [5]:
#read data for 1 crypto
#schema check/validation need to do this, create spark df using schema struct
#implement transformations
#add unique id for crypto,price,time
#split into 3 tables
#upload to postgres db

1:1: E265 block comment should start with '# '
2:1: E265 block comment should start with '# '
3:1: E265 block comment should start with '# '
4:1: E265 block comment should start with '# '
5:1: E265 block comment should start with '# '
6:1: E265 block comment should start with '# '


In [6]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG) #set to debug to capture all levels
if logger.hasHandlers():
    logger.handlers.clear()
logger.propagate = False


2:31: E261 at least two spaces before inline comment
2:32: E262 inline comment should start with '# '
6:1: W391 blank line at end of file


In [7]:
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
logger.addHandler(handler)

In [8]:
cryptos = ['BTCUSDT','ETHUSDT','LTCUSDT','BNBUSDT','DOGEUSDT']
client_minio = Minio(
        "localhost:9000",  # Make sure you're using port 9000 for the S3 API
        #minio_url,
        access_key = MINIO_USER,
        secret_key = MINIO_PASSWORD,
        secure=False  # Disable SSL if you're not using SSL certificates
        )

1:21: E231 missing whitespace after ','
1:31: E231 missing whitespace after ','
1:41: E231 missing whitespace after ','
1:51: E231 missing whitespace after ','
4:9: E265 block comment should start with '# '
5:19: E251 unexpected spaces around keyword / parameter equals
5:21: E251 unexpected spaces around keyword / parameter equals
6:19: E251 unexpected spaces around keyword / parameter equals
6:21: E251 unexpected spaces around keyword / parameter equals


In [9]:
client_minio.fget_object(bucket_name = "binancedata", 
                         object_name ="BNBUSDT/Monthly/year=2019/month=1/part-00000-c9e58e49-1c79-4e98-b1a7-f0611e7acc19.c000.snappy.parquet", 
                         file_path ="./downloaded.parquet"
                        )
#binancedata/BNBUSDT/Monthly/year=2019/month=1
#binancedata/BNBUSDT/Monthly/year=2019/month=1/part-00000-c9e58e49-1c79-4e98-b1a7-f0611e7acc19.c000.snappy.parquet

1:37: E251 unexpected spaces around keyword / parameter equals
1:39: E251 unexpected spaces around keyword / parameter equals
1:54: W291 trailing whitespace
2:37: E251 unexpected spaces around keyword / parameter equals
2:80: E501 line too long (142 > 79 characters)
2:143: W291 trailing whitespace
3:35: E251 unexpected spaces around keyword / parameter equals
4:25: E124 closing bracket does not match visual indentation
5:1: E265 block comment should start with '# '
6:1: E265 block comment should start with '# '
6:80: E501 line too long (114 > 79 characters)


In [10]:
objects = client_minio.list_objects("binancedata", prefix="BNBUSDT", recursive=True)
filenames = [obj.object_name for obj in objects]
filenames = [f for f in filenames if "_SUCCESS" not in f]

1:80: E501 line too long (84 > 79 characters)


In [12]:
'''client_minio.fget_object(bucket_name = "binancedata", 
                         object_name =filenames[0], 
                         file_path =local_file_path
                        )'''

'client_minio.fget_object(bucket_name = "binancedata", \n                         object_name =filenames[0], \n                         file_path =local_file_path\n                        )'

1:57: W291 trailing whitespace
2:52: W291 trailing whitespace


In [19]:
df_bnb = spark.read.parquet("s3a://binancedata/BNBUSDT/Monthly/year=2019/month=1/part-00000-c9e58e49-1c79-4e98-b1a7-f0611e7acc19.c000.snappy.parquet")

1:80: E501 line too long (150 > 79 characters)


In [22]:
for file in filenames:
    df = spark.read.parquet(f"s3a://binancedata/{file}")
    df_bnb = df_bnb.union(df)
                            
    
    

4:1: W293 blank line contains whitespace
5:1: W293 blank line contains whitespace
6:1: W293 blank line contains whitespace


In [27]:
df_bnb.show(30)

+-------------------+------------+------------+------------------+
|           datetime|  Open Price| Close Price|            Volume|
+-------------------+------------+------------+------------------+
|2019-01-01 00:00:00|  6.11390000|  6.22000000| 72126339.77000000|
|2019-01-01 00:00:00|  6.11390000|  6.22000000| 72126339.77000000|
|2019-10-01 01:00:00| 15.84930000| 19.90990000| 53481014.91000000|
|2019-11-01 00:00:00| 19.90870000| 15.71180000| 46441747.10000000|
|2019-12-01 00:00:00| 15.70300000| 13.71610000| 38189582.55000000|
|2019-02-01 00:00:00|  6.22500000| 10.27790000|110685041.86000000|
|2019-03-01 00:00:00| 10.27770000| 17.53760000|115379333.44000000|
|2019-04-01 01:00:00| 17.52010000| 22.03910000| 83676534.62000000|
|2019-05-01 01:00:00| 22.03000000| 32.88620000|104145121.86000000|
|2019-06-01 01:00:00| 32.91600000| 32.11400000| 93155800.74000000|
|2019-07-01 01:00:00| 32.11410000| 27.75000000| 71835825.11000000|
|2019-08-01 01:00:00| 27.73480000| 21.06040000| 43730003.73000

In [14]:
#create tmp file
#read data from 1 year and append to a spark df
#do transformations when its in spark df
#push to postgres

1:1: E265 block comment should start with '# '
2:1: E265 block comment should start with '# '
3:1: E265 block comment should start with '# '
4:1: E265 block comment should start with '# '
